<a href="https://colab.research.google.com/github/arnavkhurma/Finance-Related-Projects/blob/main/quantitative_momentum_investing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantitative Momentum Investing
Momentum Investing refers to the process of investing into stocks that have increased in price the most, over a particular period of time.

For this exploration, I will be using an equal-weighted approach in Commodities.

In [3]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import math
from scipy import stats
import xlsxwriter
# import fredapi as fa
import yfinance as yf
import datetime as dt
from datetime import timedelta
import os

In [4]:
# List of stocks
commodities = ["ZS=F", "ZC=F", "ZW=F", "SB=F",
               "CL=F", "BZ=F", "RB=F", "UGA",
               "NG=F", "GC=F", "SI=F", "HG=F"]
com_len = len(commodities)
com_len

12

In [5]:
# Function to load ticker data from Yahoo Finance
def load_data(ticker, start_date, end_date, type):
    interval = "1d"
    data = yf.Ticker(ticker).history(start=start_date, end=end_date, interval=interval)[type]
    return data.to_frame()

In [6]:
# Defining Dates and COunter
start_date = dt.date(2018, 1, 1)
end_date = dt.date.today()
counter = 0
# length_list = []
# print(start_date, end_date)

for ticker in commodities:
    current = load_data(ticker, start_date, end_date, "Open")
    counter += 1
    current.index = current.index.tz_localize(None)
    # length_list.append(str(current.shape))
    # print(current)
    # print(type(current))
    # print(current)

    # Make the directory
    raw_data = 'raw_data'
    if not os.path.exists(raw_data):
        os.makedirs(raw_data)
        print(f"Directory '{raw_data}' created.")

    # Creating a file-path and writing to the file path.
    writer = pd.ExcelWriter(os.path.join(raw_data, ticker + ".xlsx"), engine = 'xlsxwriter')
    current.to_excel(writer, header = True, index = True)
    writer.close()

# Printing out the status of files
if (counter != 1):
    print(str(counter) + "/" + str(com_len) + " files created.")
else:
    print("1 New file created.")
# print(length_list)

Directory 'raw_data' created.
12/12 files created.


In [8]:
# Calculates the price exactly one year ago. If market was not
# open, it keeps adding a day until the market was open.
def one_year_price(index_current, df, current_price):
    if ((index_current - 365) >= 0):
        # Row exists within the dataframe.
        row_one_year_ago = df.iloc[index_current - 365]
        price = float(row_one_year_ago["Open"])
        if (price != 0):
            return price
        return current_price
    else:
        # Row does not exist within the dataframe.
        return -1;

# Calculates the yearly return, given the current and old price.
def one_year_price_return(current_price, index_current, df):
    one_year_ago_price = one_year_price(index_current, df, current_price)
    if (one_year_ago_price == -1):
        return None
    else:
        return (float(current_price/one_year_price(index_current, df, current_price)) - 1.0)

In [9]:
oypr = []
dataframes = []
number = 0
# For each file in the "raw_data" directory
for fn in os.listdir(raw_data):
    number += 1
    # Create a dataframe
    df = pd.read_excel(os.path.join(raw_data, fn))
    sum_oypr = 0
    # Initialize empty lists to store calculated values
    one_year_ago_prices = []
    one_year_price_returns = []
    # For each row in the dataframe, starting from 365 up until the end
    for i in range(365, len(df)):
        current_price = float(df.loc[i, "Open"])
        current_oypr = one_year_price_return(current_price, i, df)
        old_price = one_year_price(i, df, current_price)
        # Adding current one-year price return to the total sum for file.
        sum_oypr += current_oypr
        one_year_ago_prices.append(old_price)
        one_year_price_returns.append(current_oypr)

    # Remove the first 365 entries of the dataframe since there is a lag
    df = df.iloc[365:]
    # Assign the calculated values to new columns in the DataFrame
    df["OneYearAgoPrice"] = one_year_ago_prices
    df["OneYearPriceReturn"] = one_year_price_returns
    # Calculating the average one-year price return and
    # appending it to a new list.
    average_oypr = sum_oypr / (len(df) - 365)
    oypr.append((fn[:-5], average_oypr))
    dataframes.append(df)
# print(oypr)
print(len(dataframes), number, counter, com_len)
print(dataframes[0])

<ipython-input-9-4bf46006ee6f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["OneYearAgoPrice"] = one_year_ago_prices
<ipython-input-9-4bf46006ee6f>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["OneYearPriceReturn"] = one_year_price_returns
<ipython-input-9-4bf46006ee6f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

12 12 12 12
           Date       Open  OneYearAgoPrice  OneYearPriceReturn
365  2019-06-18  14.980000        17.155001           -0.126785
366  2019-06-19  14.945000        17.080000           -0.125000
367  2019-06-20  15.469000        17.084999           -0.094586
368  2019-06-21  15.295000        17.205000           -0.111014
369  2019-06-24  15.368000        17.063999           -0.099390
...         ...        ...              ...                 ...
1422 2023-08-29  24.777000        24.975000           -0.007928
1423 2023-08-30  24.780001        25.200001           -0.016667
1424 2023-08-31  24.665001        25.410000           -0.029319
1425 2023-09-01  24.490000        25.174999           -0.027210
1426 2023-09-05  23.879999        25.195000           -0.052193

[1062 rows x 4 columns]


<ipython-input-9-4bf46006ee6f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["OneYearAgoPrice"] = one_year_ago_prices
<ipython-input-9-4bf46006ee6f>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["OneYearPriceReturn"] = one_year_price_returns


In [10]:
# Let us take the average of the last One-year Price Return and the overall Average One-Year Price Return
final_dataframe_columns = ["Ticker", "CurrentPrice", "AverageOne-YearPriceReturn",
                           "LastOne-YearPriceReturn", "NumberOfSharesToBuy"]
final_dataframe = pd.DataFrame(columns=final_dataframe_columns)
final_tickers = []
final_price = []
final_avg_oypr = []
final_current_oypr = []
final_shares = []
for i in range(len(oypr)):
    ticker = oypr[i][0]
    avg_oypr = oypr[i][1]
    shares = "N/A"
    most_recent_price = float(dataframes[i]["Open"].iloc[-1])
    most_recent_oypr = float(dataframes[i]["OneYearPriceReturn"].iloc[-1])

    # Appending values to the lists
    final_tickers.append(ticker)
    final_price.append(most_recent_price)
    final_avg_oypr.append(avg_oypr)
    final_current_oypr.append(float(dataframes[i]["OneYearPriceReturn"].iloc[-1]))
    final_shares.append(shares)

# Adding columns and respective values to the final_dataframe
final_dataframe["Ticker"] = final_tickers
final_dataframe["CurrentPrice"] = final_price
final_dataframe["LastOne-YearPriceReturn"] = final_current_oypr
final_dataframe["AverageOne-YearPriceReturn"] = final_avg_oypr
final_dataframe["NumberOfSharesToBuy"] = final_shares
print(final_dataframe)
# print(ticker, avg_oypr, shares, most_recent_oypr, most_recent_price)

   Ticker  CurrentPrice  AverageOne-YearPriceReturn  LastOne-YearPriceReturn  \
0    SI=F     23.879999                    0.257451                -0.052193   
1    ZW=F    601.250000                    0.317354                -0.462810   
2    GC=F   1951.500000                    0.223844                 0.016036   
3    ZC=F    488.000000                    0.402087                -0.355137   
4    SB=F     26.129999                    0.338228                 0.351785   
5    BZ=F     89.980003                    0.365287                -0.229953   
6     UGA     72.500000                    0.698788                 0.222597   
7    HG=F      3.837500                    0.234188                -0.180284   
8    RB=F      2.586400                    0.492274                -0.224002   
9    NG=F      2.584000                    0.613282                -0.477769   
10   ZS=F   1377.000000                    0.330692                -0.186531   
11   CL=F     86.720001                 

In [11]:
# Removing the low momentum stocks, since we need to indentify the top 10 highest momentum stocks.
# Since we have about 35 entries in our final_dataframe, we will be removing the bottom 25.
top = 3
# Sorting the DataFrame by One-Year Price Return
final_dataframe.sort_values("LastOne-YearPriceReturn", ascending=False, inplace = True)
# Updating the final_dataframe to only the top 10 values
final_dataframe = final_dataframe[:top]
final_dataframe.reset_index(inplace = True)

# Writing to an excel file.
writer = pd.ExcelWriter(os.path.join("final_dataframe.xlsx"), engine = 'xlsxwriter')
final_dataframe.to_excel(writer, header = True, index = True)
writer.close()
# Here, inplace takes the place of the original dataframe. ascending = false means that
# it will be sorted highest to lowest (descending order)
final_dataframe

,index,Ticker,CurrentPrice,AverageOne-YearPriceReturn,LastOne-YearPriceReturn,NumberOfSharesToBuy
0,4,SB=F,26.129999,0.338228,0.351785,N/A
1,6,UGA,72.500000,0.698788,0.222597,N/A
2,2,GC=F,1951.500000,0.223844,0.016036,N/A


In [12]:
# Calculating the number of shares to buy
def calculate_number_of_shares():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")
    try:
        float(portfolio_size)
    except ValueError:
        print("That is not a number, please try again!")
        portfolio_size = input("Enter the size of your portfolio: ")

calculate_number_of_shares()
print(portfolio_size)
# Sample: $1,300,000

Enter the size of your portfolio: 1300000
1300000


In [13]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, "NumberOfSharesToBuy"] = math.floor(position_size/final_dataframe.loc[i, "CurrentPrice"])
final_dataframe
# print(position_size)

,index,Ticker,CurrentPrice,AverageOne-YearPriceReturn,LastOne-YearPriceReturn,NumberOfSharesToBuy
0,4,SB=F,26.129999,0.338228,0.351785,16583
1,6,UGA,72.500000,0.698788,0.222597,5977
2,2,GC=F,1951.500000,0.223844,0.016036,222
